In [67]:
import torch
import random
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [51]:
DECK = 4 * (list(range(2, 12)) + [10, 10, 10])

class Value(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(8, 12)
        self.lin2 = nn.Linear(12, 12)
        self.lin3 = nn.Linear(12, 12)
        self.lin4 = nn.Linear(12, 1)
        

    def forward(self, x):
        x = self.lin1(x)
        x = torch.relu(x)
        x = self.lin2(x)
        x = torch.relu(x)
        x = self.lin3(x)
        x = torch.relu(x)
        return self.lin4(x)
    
class Policy(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(8, 12)
        self.lin2 = nn.Linear(12, 12)
        self.lin3 = nn.Linear(12, 12)
        self.lin4 = nn.Linear(12, 4) # double, split, stand, hit
        self.softmax = torch.softmax

    def forward(self, x):
        x = self.lin1(x)
        x = torch.relu(x)
        x = self.lin2(x)
        x = torch.relu(x)
        x = self.lin3(x)
        x = torch.relu(x)
        x = self.lin4(x)
        return self.softmax(x, dim = -1)


In [3]:
def get_bet(count):
    return 10.

def is_soft(c1, c2):
    return c1 == 11 or c2 == 11

def get_score(hand, c, soft):
    """returns the score and its softness given the existing hand and a new card"""
    if c != 11 and not soft:
        return False, hand + c

    if c == 11 and not soft: # if valid with 1 or 11, then is still soft (and add 11). else add 1 only
        if hand + 11 <= 21: # and by extension vaild with hand + 1
            return True, hand + 11

        return False, hand + 1

    if c != 11 and soft: # if total > 21, then subtract 10 and it's no longer soft. else, add the card and it's still soft 
        if hand + c > 21:
            return False, hand + c - 10

        return True, hand + c

    # else c == 11 and it's soft
    # my argument: we add one and it remains soft since by construction, a hand cannot be soft
    # unless it is of total at least 11
    return True, hand + 1


In [4]:
### TROUBLESHOOTING GET SCORE
#for p in range(2, 21): # isch guet
#    for c in range(2, 12):
#        s, np = get_score(p, c, False)
#        print(p, 'plus', c, 'is', np, s)

#for p in range(11, 21): # i think it's good but i don't want to check rn :)
#    for c in range(2, 12):
#        s, np = get_score(p, c, True)
#        print(p, 'plus', c, 'is', np, s)

In [5]:
def blackjack(c1, c2):
    return (c1 == 11 and c2 == 10) or (c1 == 10 and c2 == 11)

In [97]:
def g(A, epsilon):
    return (1 + epsilon) * A * (A >= 0) + (1 - epsilon) * A * (A < 0)

In [98]:
def do_game(policy, verbose = False):
    """
    does rounds of blackjack until the shoe is sufficiently low (here defined to be 20 cards) 
    """
    shoe = 8 * DECK
    random.shuffle(shoe)

    count = 0.
    winnings = 0.

    S = []
    P = []
    A = []
    R = []

    while len(shoe) > 20:
        if verbose:
            print('\n=== NEW ROUND ===')
            
        bet = get_bet(count)
        s, p, a, reward = round(shoe, policy, bet, verbose)

        S += s
        P += p
        A += a
        R += [reward for _ in s]

        winnings += reward

        #print(len(s), len(p), len(a))

    return S, P, A, R, winnings

def round(shoe, policy, bet, verbose = False):
    """
    one round of blackjack
    """

    # initialize lists of states and actions to be undertaken
    states = []
    probabilities = []
    actions = []

    penalty = -1000.
    bet2 = 0.
    
    # deal player cards
    c1 = shoe.pop(0)
    c2 = shoe.pop(0)

    player = 0.
    dealer = 0.
    
    splittable = c1 == c2

    soft, player = get_score(player, c1, False)
    soft, player = get_score(player, c2, soft)

    # deal dealer cards
    d1 = shoe.pop(0)
    d2 = shoe.pop(0)

    soft_dealer, dealer = get_score(dealer, d1, False)

    # initiallize state vector
    # splittable | doublable | soft | score | doublable2 | soft2 | score2 | dealer upcard
    state = torch.tensor(
        [
            splittable, # can split or not
            True, # can double 
            soft, # soft or not
            player, # player score 
            False, False, 0.,
            dealer
        ])


    # check for blackjacks
    if blackjack(d1, d2) and not blackjack(c1, c2): # dealer, not player
        if verbose:
            print('Dealer blackjack')
            print('Payout:', -1 * bet)
            
        return states, probabilities, actions, -1 * bet

    elif blackjack(d1, d2) and blackjack(c1, c2): # both
        if verbose:
            print('Double blackjack')
            print('Payout:', 0.)
            
        return states, probabilities, actions, 0.

    elif blackjack(c1, c2):
        if verbose:
            print('Blackjack!')
            print('Payout:', 1.5 * bet)
        
        return states, probabilities, actions, 1.5 * bet

    # print what's up
    if verbose:
        print('Player:', 'soft' if soft else '', player, '| Dealer:', d1)


    # player gets to 21 by doing actions
    while state[3] < 21: 

        states.append(state)
        
        # get action - 0 stand, 1 hit, 2 double, 3 split
        probs = policy(state).detach()
        action = torch.distributions.Categorical(probs = probs).sample().item()

        probabilities.append(probs)
        actions.append(action)

        if verbose:
            print('Action:', ['stand', 'hit', 'double', 'split'][action])

        if action == 0:
            break

        if action == 1:
            c = shoe.pop(0)
            state[2], state[3] = get_score(state[3], c, state[2])

            # update so no splits or doubles
            state[0] = False
            state[1] = False

            if verbose:
                print('Player:', 'soft' if state[2] else '', state[3].item())

        if action == 2:
            if not state[1]: # if we doubled when we were not permitted
                if verbose:
                    print('Invalid!')

                return states, probabilities, actions, penalty

            # else it's just the same as above
            c = shoe.pop(0)
            state[2], state[3] = get_score(state[3], c, state[2])

            # update so no splits or doubles
            state[0] = False
            state[1] = False

            # double the bet
            bet *= 2

            if verbose:
                print('Player:', 'soft' if state[2] else '', state[3].item())

            break

        if action == 3:
            if not state[0]:
                if verbose:
                    print('Invalid!')

                return states, probabilities, actions, penalty

            # can no longer split
            state[0] = False

            # redistribute cards
            if state[2] and state[3] == 12: # if double aces
                state[3] = 11.
                state[6] = 11.

                state[2] = True
                state[5] = True

            else: # otherwise it's simplly half the score
                state[3] /= 2
                state[6] = state[3]

                state[2] = False
                state[5] = False

            # add two new cards
            c = shoe.pop(0)
            state[2], state[3] = get_score(state[3], c, state[2])

            c = shoe.pop(0)
            state[5], state[6] = get_score(state[6], c, state[5])

            # update doubleables
            state[1] = True
            state[4] = True

            # instantiate the second bet
            bet2 = 1. * bet

            if verbose:
                print('Player:', 'soft' if state[2] else '', state[3].item(), '| Player 2:', 'soft' if state[5] else '', state[6].item())


    # see if the second hand must be processed
    if state[6] != 0:
        while state[6] < 21:

            states.append(state)
            
            probs = policy(state).detach()
            action = torch.distributions.Categorical(probs = probs).sample().item()

            probabilities.append(probs)
            actions.append(action)
    
            if verbose:
                print('Action 2:', ['stand', 'hit', 'double', 'split'][action])
    
            if action == 0:
                break
    
            if action == 1:
                c = shoe.pop(0)
                state[5], state[6] = get_score(state[6], c, state[5])
    
                # update so no doubles
                state[4] = False
    
                if verbose:
                    print('Player 2:', 'soft' if state[5] else '', state[6].item())
    
            if action == 2:
                if not state[4]: # if we doubled when we were not permitted
                    if verbose:
                        print('Invalid!')
    
                    return states, probabilities, actions, penalty
    
                # else it's just the same as above
                c = shoe.pop(0)
                state[5], state[6] = get_score(state[6], c, state[5])
    
                # update so no doubles
                state[4] = False
                bet2 *= 2
    
                if verbose:
                    print('Player 2:', 'soft' if state[5] else '', state[6].item())
    
                break
    
            if action == 3:
                if verbose:
                    print('Invalid!')
    
                return states, probabilities, actions, penalty


    #states.append(state)


    # we may not have to process the dealer hand
    # (1) hand 1 busts and hand 2 doesn't exist ; (2) both hands exist, and both bust
    if state[3] > 21 and state[6] == 0:
        if verbose:
            print('Bust. Payout:', -1 * bet)

        soft_dealer, dealer = get_score(dealer, d2, soft_dealer)
        return states, probabilities, actions, -1 * bet

    if state[3] > 21 and state[6] > 21:
        if verbose:
            print('Both hands bust. Payout:', -1 * (bet + bet2))

        soft_dealer, dealer = get_score(dealer, d2, soft_dealer)
        return states, probabilities, actions, -1 * (bet + bet2)

    
    # otherwise, we must process the dealer hand
    soft_dealer, dealer = get_score(dealer, d2, soft_dealer)

    while dealer < 17:
        d = shoe.pop(0)
        soft_dealer, dealer = get_score(dealer, d, soft_dealer)

    if verbose:
        print('Dealer:', dealer)

    
    # check for dealer bust
    if dealer > 21:
        if verbose:
            print('Player victory. Payout:', (2 * (state[3].item() <= 21) - 1) * bet + (2 * (state[6].item() <= 21) - 1) * bet2)

        return states, probabilities, actions, ((2 * (state[3].item() <= 21) - 1) * bet) + ((2 * (state[6].item() <= 21) - 1) * bet2)


    # do comparisons
    # only first hand ~OR~ first hand and second hand bust
    if state[3] <= 21 and (state[6] == 0 or state[6] > 21): # then it's win loss or tie (already accountued for bust case)
        if state[3] > dealer:
            if verbose:
                print('Payout:', bet - bet2)

            return states, probabilities, actions, bet - bet2

        elif state[3] == dealer:
            if verbose:
                print('Payout:', 0. - bet2)

            return states, probabilities, actions, 0. - bet2

        else: 
            if verbose:
                print('Payout:', -1 * bet - bet2)

            return states, probabilities, actions, -1 * bet - bet2

    # first hand bust, second hand valid
    if state[3] > 21 and state[6] <= 21:
        if state[6] > dealer:
            if verbose:
                print('Payout:', bet2 - bet)

            return states, probabilities, actions, bet - bet2

        elif state[6] == dealer:
            if verbose:
                print('Payout:', 0. - bet)

            return states, probabilities, actions, 0. - bet

        else: 
            if verbose:
                print('Payout:', -1 * bet2 - bet)

            return states, probabilities, actions, -1 * bet2 - bet

    # both hands valid
    if state[3] == dealer:
        if state[6] == dealer:
            if verbose:
                print('Payout:', 0.)
                
            return states, probabilities, actions, 0.
        # else we won or lost
        if verbose:
            print('Payout:', bet2 * (2 * (state[6] > dealer).item() - 1))

        return states, probabilities, actions, bet2 * (2 * (state[6] > dealer).item() - 1)

    elif state[6] == dealer:
        # then just account for first hand, already accounted for equality case
        if verbose:
            print('Payout:', bet * (2 * (state[3] > dealer) - 1).item())
            
        return states, probabilities, actions, bet * (2 * (state[3] > dealer) - 1).item()

    # else both are not ties and eligible for consideration
    if verbose:
        print('Payout:', bet * (2 * (state[3] > dealer) - 1).item() + (bet2 * (2 * (state[6] > dealer) - 1).item()))
    return states, probabilities, actions, bet * (2 * (state[3] > dealer) - 1).item() + (bet2 * (2 * (state[6] > dealer) - 1)).item()


In [99]:
policy = Policy()

do_game(policy, verbose = True)


=== NEW ROUND ===
Player: soft 19.0 | Dealer: 10
Action: hit
Player:  16.0
Action: split
Invalid!

=== NEW ROUND ===
Player:  11.0 | Dealer: 10
Action: hit
Player:  13.0
Action: hit
Player:  19.0
Action: stand
Dealer: 20.0
Payout: -10.0

=== NEW ROUND ===
Player:  11.0 | Dealer: 4
Action: hit
Player:  19.0
Action: double
Invalid!

=== NEW ROUND ===
Player:  16.0 | Dealer: 5
Action: hit
Player:  20.0
Action: stand
Dealer: 22.0
Player victory. Payout: 10.0

=== NEW ROUND ===
Player:  19.0 | Dealer: 11
Action: split
Invalid!

=== NEW ROUND ===
Player:  13.0 | Dealer: 10
Action: hit
Player:  17.0
Action: hit
Player:  22.0
Bust. Payout: -10.0

=== NEW ROUND ===
Player:  20.0 | Dealer: 9
Action: hit
Player:  27.0
Bust. Payout: -10.0

=== NEW ROUND ===
Player:  16.0 | Dealer: 10
Action: hit
Player:  23.0
Bust. Payout: -10.0

=== NEW ROUND ===
Player: soft 14.0 | Dealer: 9
Action: double
Player: soft 15.0
Dealer: 19.0
Payout: -20.0

=== NEW ROUND ===
Player:  7.0 | Dealer: 10
Action: split
In

([tensor([ 0.,  0.,  0., 16.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 16.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 19.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 19.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 19.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 19.,  0.,  0.,  0.,  4.]),
  tensor([ 0.,  0.,  0., 19.,  0.,  0.,  0.,  4.]),
  tensor([ 0.,  0.,  0., 20.,  0.,  0.,  0.,  5.]),
  tensor([ 0.,  0.,  0., 20.,  0.,  0.,  0.,  5.]),
  tensor([ 0.,  1.,  0., 19.,  0.,  0.,  0., 11.]),
  tensor([ 0.,  0.,  0., 22.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 22.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 27.,  0.,  0.,  0.,  9.]),
  tensor([ 0.,  0.,  0., 23.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  1., 15.,  0.,  0.,  0.,  9.]),
  tensor([ 0.,  1.,  0.,  7.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 22.,  0.,  0.,  0., 10.]),
  tensor([ 0.,  0.,  0., 16.,  0.,  0.,  0.,  6.]),
  tensor([ 0.,  0.,  0., 16.,  0.,  0.,  0.,  6.]),
  tensor([ 0

In [116]:
def train(policy, value, games = 100, epochs = 10):

    optimizer_value = Adam(value.parameters(), lr = 1e-3)
    optimizer_policy = Adam(policy.parameters(), lr = 1e-3)

    batch_size = 20
    W = []

    for epoch in range(epochs):

        S = []
        P = []
        A = []
        R = []

        winnings = 0.

        # generate games
        for _ in range(games):

            s, p, a, r, w = do_game(policy, verbose = False)

            S += s
            P += p
            A += a
            R += r
            
            winnings += w

        # tensorize things for easy computation + advantage estimation        
        S = torch.stack(S)
        P = torch.stack(P)
        Ai = torch.tensor(A)
        A = Ai.reshape(-1, 1)
        R = torch.tensor(R).reshape(-1, 1)

        V = R - value(S).detach()

        dataset = DataLoader(TensorDataset(S, P, Ai, A, R, V), batch_size = batch_size)
        
        # policy training loop        
        for _ in range(500):
            loss = 0.

            for s, p, ai, a, r, v in dataset:
                N = s.shape[0]

                optimizer_policy.zero_grad()
                probs = policy(s)
                
                first_argument = v * probs[torch.arange(N), ai].reshape(-1, 1) / p
                second_argument = g(v, 1e-2)
                
                li = -1 * torch.sum(torch.min(first_argument, second_argument)) # multiply by negative 1 so it goes up
                
                li.backward()
                optimizer_policy.step()

                loss += li.detach()

            #print(loss)


        # value training loop
        for _ in range(500):
            loss = 0.

            for s, p, ai, a, r, v in dataset:
                optimizer_value.zero_grad()
                
                val = value(s)
                li = torch.mean((val - r)**2)
                
                li.backward()
                optimizer_value.step()

                loss += li.detach()

            #print(loss)


        # print the important things
        print('=== Epoch', epoch, '| Winnings:', winnings, '===')
        W.append(winnings)

    return W

        
  

In [ ]:
value = Value()
policy = Policy()

train(policy, value)